In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/

/content/drive/My Drive


In [ ]:
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import KNNBasic,  KNNWithMeans, KNNBaseline
from surprise.model_selection import KFold
from surprise import Reader
# from lenskit.crossfold import partition_users, SampleFrac
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from collections import defaultdict

In [ ]:
df = pd.read_csv('df_yelp_200K.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (36) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data = df[[ 'review_id', 'user_id', 'business_id', 'name_y', 'name_x', 'review_stars']]
data.columns = ['review_id','user_id','business_id','Restaurant Name', 'User Name', 'Rating']
data.head()

,review_id,user_id,business_id,Restaurant Name,User Name,Rating
0,G1rbhEyMUVnFIq1kAT8jCg,nIJD_7ZXHq-FX8byPMOkMQ,J8GuDrqeLH1xMpSpeEixYA,Biscuits,Nicole,2.0
1,OtH9uCUr5e2Kz12F1LboXA,pu_AQig2fw40PshvtgONPQ,J8GuDrqeLH1xMpSpeEixYA,Biscuits,Stephanie,2.0
2,UcQhHkWR8YfIOWeLzjflNA,xUhKi3p2BRTEbTHtJz-Hhg,J8GuDrqeLH1xMpSpeEixYA,Biscuits,S.Ellen,4.0
3,IJj3by6GI9jV8d5wI46Taw,Vv1jaDcK9fdaqGwODB0AcA,J8GuDrqeLH1xMpSpeEixYA,Biscuits,Bruce,3.0
4,ynCKeNNCL76ws3W3AtbAKQ,ZefHQwjj3b7mxi46oKik-g,J8GuDrqeLH1xMpSpeEixYA,Biscuits,Patrick,3.0


In [ ]:
np.random.seed(111)
msk = np.random.rand(len(data)) < 0.8
df_train = data[msk]
df_test = data[~msk]

In [ ]:
reader = Reader(rating_scale=(1, 5))

In [ ]:
kf = KFold(n_splits=5, random_state=111)
algo = KNNBasic()
best_algo = None
best_rmse = 1000.0
best_pred = None
j = 0

for i in range(10):
    data_train_subset = df_train.iloc[j:j+len(df_train)//10]
    print(data_train_subset.shape)
    data_train_subset = Dataset.load_from_df(data_train_subset[['user_id', 'business_id', 'Rating']], reader)
    for trainset, testset in kf.split(data_train_subset):
        algo.fit(trainset)
        predictions = algo.test(testset)
        rmse = accuracy.rmse(predictions, verbose=True)
        mae = accuracy.mae(predictions, verbose=True)
        
        if rmse < best_rmse:
            best_algo = algo
            best_pred = predictions
    j = j+len(df_train)//10

(18493, 6)
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.2604
MAE:  1.0031
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.2562
MAE:  0.9968
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.2878
MAE:  1.0209
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.2675
MAE:  1.0037
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.2599
MAE:  0.9972
(18493, 6)
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.2090
MAE:  0.9008
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.1864
MAE:  0.9020
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.1981
MAE:  0.9043
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.1905
MAE:  0.8864
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.1793
M

In [ ]:
kf_test = KFold(n_splits=2, random_state=111)
data_test = Dataset.load_from_df(df_test[['user_id', 'business_id', 'Rating']], reader)
test_rmse = 0
test_mae = 0

for _,testset in kf_test.split(data_test):
    predictions = best_algo.test(testset)
    test_rmse += accuracy.rmse(predictions, verbose=True)
    test_mae += accuracy.mae(predictions, verbose=True)
    
print("RMSE on test set = ", test_rmse/2)
print("MAE on test set = ", test_mae/2)

RMSE: 1.4136
MAE:  1.2593
RMSE: 1.4154
MAE:  1.2624
RMSE on test set =  1.4145107463963207
MAE on test set =  1.260844474429252


In [ ]:
# Fitting with splits 5
kf = KFold(n_splits=5)
sim_options = {'name':'cosine'}
algo = KNNWithMeans(sim_options = sim_options)
best_algo = None
best_rmse = 1000.0
best_pred = None
j = 0

for i in range(10):
    data_train_subset = df_train.iloc[j:j+len(df_train)//10]
    data_train_subset = Dataset.load_from_df(data_train_subset[['user_id', 'business_id', 'Rating']], reader)
    for trainset, testset in kf.split(data_train_subset):
        algo.fit(trainset)
        predictions = algo.test(testset)
        rmse = accuracy.rmse(predictions, verbose=True)
        mae = accuracy.mae(predictions, verbose=True)
        
        if rmse < best_rmse:
            best_algo = algo
            best_pred = predictions
    j = j+len(df_train)//10

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.2648
MAE:  1.0024
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.2840
MAE:  1.0177
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.2635
MAE:  0.9986
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.3008
MAE:  1.0270
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.2832
MAE:  1.0178
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.1913
MAE:  0.8993
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.2155
MAE:  0.9121
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.2186
MAE:  0.9142
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.1925
MAE:  0.8915
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 

In [ ]:
kf_test = KFold(n_splits=2, random_state=111)
data_test = Dataset.load_from_df(df_test[['user_id', 'business_id', 'Rating']], reader)
test_rmse = 0
test_mae = 0

for _,testset in kf_test.split(data_test):
    predictions = best_algo.test(testset)
    test_rmse += accuracy.rmse(predictions, verbose=True)
    test_mae += accuracy.mae(predictions, verbose=True)
    
print("RMSE on test set = ", test_rmse/2)
print("MAE on test set = ", test_mae/2)

RMSE: 1.4145
MAE:  1.2569
RMSE: 1.4192
MAE:  1.2624
RMSE on test set =  1.4168308801755463
MAE on test set =  1.2596837386932362


In [ ]:
# Fitting with splits 5
kf = KFold(n_splits=5)
sim_options = {'name':'msd'}
algo = KNNWithMeans(sim_options = sim_options)
best_algo = None
best_rmse = 1000.0
best_pred = None
j = 0

for i in range(10):
    data_train_subset = df_train.iloc[j:j+len(df_train)//10]
    data_train_subset = Dataset.load_from_df(data_train_subset[['user_id', 'business_id', 'Rating']], reader)
    for trainset, testset in kf.split(data_train_subset):
        algo.fit(trainset)
        predictions = algo.test(testset)
        rmse = accuracy.rmse(predictions, verbose=True)
        mae = accuracy.mae(predictions, verbose=True)
        
        if rmse < best_rmse:
            best_algo = algo
            best_pred = predictions
    j = j+len(df_train)//10

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.2664
MAE:  1.0097
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.2975
MAE:  1.0155
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.2733
MAE:  1.0127
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.2578
MAE:  1.0010
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.3091
MAE:  1.0383
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.2227
MAE:  0.9156
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.1950
MAE:  0.9021
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.1759
MAE:  0.8900
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.2038
MAE:  0.8980
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.2157
MAE:  0.9151
Computing 

In [ ]:
kf_test = KFold(n_splits=2, random_state=111)
data_test = Dataset.load_from_df(df_test[['user_id', 'business_id', 'Rating']], reader)
test_rmse = 0
test_mae = 0

for _,testset in kf_test.split(data_test):
    predictions = best_algo.test(testset)
    test_rmse += accuracy.rmse(predictions, verbose=True)
    test_mae += accuracy.mae(predictions, verbose=True)
    
print("RMSE on test set = ", test_rmse/2)
print("MAE on test set = ", test_mae/2)

RMSE: 1.4153
MAE:  1.2585
RMSE: 1.4196
MAE:  1.2632
RMSE on test set =  1.4174179496932235
MAE on test set =  1.2608683581383935
